# RSI_Hedge_v2.7

## Setup

### Packages

In [3]:
from utils.packages import *

### Inputs and Parameters

#### Read Yaml files

In [4]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currs = yaml.load(temp_file)        

## Functions

In [3]:
#--------------------------------------------------------------------------------------------------------------------------
def stream_speed_check():
    for i, resp in enumerate(response_stream): 

        if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
            continue

        # Date and Data ---------------------------------------------
        get_date_time(resp)
        get_prices(resp)
        l = ['ts_date_val', 'ts_time_val', 'price_ask', 'price_bid'] 

        for j, val in enumerate(l):
            print(f"{val} : {data[val]}")

        display.clear_output(wait = True)
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get timestamp of the price and segregate it
def get_date_time(resp):
    global data
    time_stamp = resp['time']
    data['ts_date_val'], full_time = time_stamp.split(sep = 'T')
    data['ts_time_val'], _ = full_time.split(sep = '.')

    data['ts_date_val'] = datetime.strptime(data['ts_date_val'], '%Y-%m-%d')
    data['ts_time_val'] = datetime.strptime(data['ts_time_val'], '%H:%M:%S')

    tot_ts = datetime.combine(datetime.date(data['ts_date_val']), datetime.time(data['ts_time_val']))
    tot_ts = tot_ts + timedelta(hours=8)
    data['tot_ts'] = tot_ts.strftime("%Y-%b-%d, %I:%M:%S (%p)")
    
    #data['ts_date_val'] = tot_ts.date()
    #data['ts_time_val'] = tot_ts.time()
    
    t2 = datetime.now()    
    data['time_diff'] = (t2 - tot_ts).total_seconds()
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get bid and ask prices
def get_prices(resp):    
    global data
    data['price_bid'] = float(resp['bids'][0]['price'])    
    data['price_ask'] = float(resp['asks'][0]['price'])
    data['price_spread'] = data['price_ask'] - data['price_bid']
    data['price_tick'] = (data['price_ask'] + data['price_bid']) / 2
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def before_rsi_len():
    global data
    data['list_tick'].append(data['price_tick'])

    if len(data['list_tick']) == 1:
        data['list_up'].append(0)
        data['list_down'].append(0)
        data['list_AvgGain'].append(0)
        data['list_Avgloss'].append(0)
        data['list_RS'].append(0)
        data['list_RSI'].append(0)
    elif len(data['list_tick']) > 1:
        old_price = data['list_tick'][len(data['list_tick'])-2]
        new_price = data['price_tick']
        if new_price > old_price:
            data['list_up'].append(new_price - old_price)
            data['list_down'].append(0)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi_val'])            
        elif old_price > new_price:
            data['list_up'].append(0)
            data['list_down'].append(old_price - new_price)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi_val'])            
        elif old_price == new_price:
            data['list_up'].append(0)
            data['list_down'].append(0)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi_val'])            
    display.clear_output(wait = True)
    print(f"Building RSI Tick list : {len(data['list_tick'])}")
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def after_rsi_len():
    global data
    data['list_up'].popleft()
    data['list_down'].popleft()
    data['list_AvgGain'].popleft()
    data['list_Avgloss'].popleft()
    data['list_RS'].popleft()
    data['list_RSI'].popleft()
    data['list_tick'].popleft()
    data['list_tick'].append(data['price_tick'])

    old_price = data['list_tick'][len(data['list_tick'])-2]
    new_price = data['price_tick']
    if new_price > old_price:
        data['list_up'].append(new_price - old_price)
        data['list_down'].append(0)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi_val'])
    elif old_price > new_price:
        data['list_up'].append(0)
        data['list_down'].append(old_price - new_price)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi_val'])
    elif old_price == new_price:
        data['list_up'].append(0)
        data['list_down'].append(0)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi_val'])            

    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def check_for_open_orders():
    global data
    
    request_position_data = positions.OpenPositions(accountID=accountID)
    data['positions_info'] = api.request(request_position_data)

    if len(data['positions_info']['positions']) == 0:
        data['order_current_open'] = False

    elif len(data['positions_info']['positions']) == 1:
        data['positions_long'] = abs(int(data['positions_info']['positions'][0]['long']['units']))
        data['positions_short'] = abs(int(data['positions_info']['positions'][0]['short']['units']))

        if data['positions_long'] >= 1 and data['positions_short'] == 0:
            data['order_current_open'] = 'long'
        elif data['positions_long'] == 0 and data['positions_short'] >= 1:
            data['order_current_open'] = 'short'                        
        elif data['positions_long'] >= 1 and data['positions_short'] >= 1:
            data['order_current_open'] = 'both'            

    return()
#==========================================================================================================================    

#--------------------------------------------------------------------------------------------------------------------------
def check_rsi_status():
    global data
    
    if not data['order_current_open']:
        if data['rsi_val'] <= data['rsi_os']:
            data['rsi_status'] = 'OS'
        elif data['rsi_val'] >= data['rsi_ob']:
            data['rsi_status'] = 'OB'
        else:
            data['rsi_status'] = None
            data['order_create'] = None
    
    #rsi_peak_check()
    rsi_direct()
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def rsi_direct():
    global data
    if not data['order_current_open']:
        if data['rsi_status'] == 'OB':
            data['order_create'] = 'short'
                
        if data['rsi_status'] == 'OS':
            data['order_create'] = 'long'
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def rsi_peak_check():
    global data
    if not data['order_current_open']:
        if data['rsi_status'] == 'OB':
            data['rsi_max'] = max(data['rsi_max'], data['rsi_val'])
            if data['rsi_val'] < data['rsi_max']:
                data['order_create'] = 'short'
                data['rsi_max'] = 0
                
        if data['rsi_status'] == 'OS':
            data['rsi_min'] = min(data['rsi_min'], data['rsi_val'])
            if data['rsi_val'] > data['rsi_min']:
                data['order_create'] = 'long'
                data['rsi_min'] = 100                
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def make_order():
    global data
    if not data['order_current_open']:
        if data['order_create'] == 'long' and data['slope_type'] != 'short':            
            data['order_val'] = data['order_num'] * 1
                        
            data['price_stop'] = max(min(data['slope_bids']), (data['price_ask'] - data['pip_stop_loss'])) - data['pip_size']
            
            stopLossOnFill = StopLossDetails(price=data['price_stop'])
                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)

            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['response_order'] = api.request(order_request_data)
            check_for_open_orders()


        if data['order_create'] == 'short' and data['slope_type'] != 'long':
            data['order_val'] = data['order_num'] * -1
            
            data['price_stop'] = min(max(data['slope_asks']), (data['price_bid'] + data['pip_stop_loss'])) + data['pip_size']
            stopLossOnFill = StopLossDetails(price=data['price_stop'])

            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['response_order'] = api.request(order_request_data)
            check_for_open_orders()
    
    
    return()
#==========================================================================================================================

def close_long_orders():
    global data
    data_long = {"longUnits": "ALL"}
    order_close_data = positions.PositionClose(accountID=accountID,
                                instrument=data['instrument'],
                                data=data_long)
    data['response_close'] = api.request(order_close_data)
    return()

def close_short_orders():
    global data
    data_short = {"shortUnits": "ALL"}
    order_close_data = positions.PositionClose(accountID=accountID,
                                instrument=data['instrument'],
                                data=data_short)
    data['response_close'] = api.request(order_close_data)
    return()


def try_close_both():
    global data
    try:
        close_long_orders()
    except:
        pass

    try:
        close_short_orders()
    except:
        pass

    return()

#--------------------------------------------------------------------------------------------------------------------------
def take_profit():
    global data
    
    if data['order_current_open'] == 'both':
        data['price_order_ask'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['long_profit_val'] = data['price_bid'] - data['price_order_ask']
        data['price_order_bid'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['short_profit_val'] = data['price_order_bid'] - data['price_ask']
        data['profit_val_both'] =  data['long_profit_val'] + data['short_profit_val']
        
        if data['profit_val_both'] >= data['pip_take_profit']:
            close_long_orders()
            close_short_orders()
            reset_data()
            check_for_open_orders()
            
                
    
    if data['order_current_open'] == 'long' or data['order_current_open'] == 'both':
        data['price_order_ask'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['long_profit_val']      = data['price_bid'] - data['price_order_ask']
        
        data['long_max_profit']      = max(data['long_max_profit'], data['long_profit_val'])
        data['long_buffer_profit']   = data['long_max_profit'] - data['pip_take_profit']
        data['long_buffer_profit']   = max(data['long_buffer_profit'], data['pip_take_profit'], (data['long_max_profit'] * 0.3))
        if data['long_buffer_profit'] >= data['long_profit_val'] >= data['pip_take_profit']:
            close_long_orders()
            reset_data()
            check_for_open_orders()
                
                
    if data['order_current_open'] == 'short' or data['order_current_open'] == 'both':
        data['price_order_bid'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['short_profit_val']      = data['price_order_bid'] - data['price_ask']
        
        data['short_max_profit']      = max(data['short_max_profit'], data['short_profit_val'])
        data['short_buffer_profit']   = data['short_max_profit'] - data['pip_take_profit']
        data['short_buffer_profit']   = max(data['short_buffer_profit'], data['pip_take_profit'], (data['short_max_profit'] * 0.3))
        if data['short_buffer_profit'] >= data['short_profit_val'] >= data['pip_take_profit']:
            close_short_orders()
            reset_data() 
            check_for_open_orders()
    
    return()
#==========================================================================================================================
          
    
#--------------------------------------------------------------------------------------------------------------------------
def make_hedge():
    if data['order_current_open'] == 'long':
        data['price_order_ask'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['long_profit_val']      = data['price_bid'] - data['price_order_ask']
        if data['long_profit_val'] <= -data['pip_start_hedge']:
            data['order_val'] = data['order_num'] * -1
            #data['price_stop'] = data['price_bid'] + data['pip_stop_loss']
            data['price_stop'] = max(data['slope_asks'])
            stopLossOnFill = StopLossDetails(price=data['price_stop'])
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            data['response_order'] = api.request(order_request_data)
            
    if data['order_current_open'] == 'short':
        data['price_order_bid'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['short_profit_val']      = data['price_order_bid'] - data['price_ask']
        if data['short_profit_val'] <= -data['pip_start_hedge']:
            data['order_val'] = data['order_num'] * 1            
            #data['price_stop'] = data['price_ask'] - data['pip_stop_loss']
            data['price_stop'] = min(data['slope_bids'])
            stopLossOnFill = StopLossDetails(price=data['price_stop'])                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)            
            data['response_order'] = api.request(order_request_data)  
            
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def dict_to_tempdf():
    global data
    data_rec = pd.DataFrame()    
    ordered_col_list = ['iter', 'start_ts', 'tot_ts', 'time_diff', 'icount', 'instrument', 'pip_size', 
                        'order_num', 'val_profit_pip', 'val_stop_loss', 'val_start_hedge', 'rsi_len',                         
                        'rsi_ob', 'rsi_os', 'iter_limit_flag', 'iter_max', 'rsi_status', 'rsi_val', 'slope_tick', 'slope_type',
                        'order_create', 'order_current_open', 'price_tick', 'price_ask', 'price_bid', 
                        'price_order_ask', 'price_order_bid', 'price_stop', 'positions_long', 
                        'long_buffer_profit', 'long_max_profit', 'long_profit_val', 'positions_short', 
                        'short_buffer_profit', 'short_max_profit', 'short_profit_val', 'profit_val_both', 
                        'pip_take_profit', 'pip_stop_loss', 'pip_start_hedge', 'rsi_max', 'rsi_min', 
                        'price_spread', 'positions_info', 'response_order', 'response_close', 'list_tick', 
                        'list_up', 'list_down', 'list_AvgGain', 'list_Avgloss', 'list_RS', 'list_RSI']

    data_rec = data_rec.append(data, ignore_index=True)
    data_rec = data_rec[ordered_col_list]
    
    return(data_rec)
#==========================================================================================================================


def before_slope_len():
    global data
    data['slope_ticks'].append(data['price_tick'])  
    data['slope_asks'].append(data['price_ask'])  
    data['slope_bids'].append(data['price_bid'])  
        
    display.clear_output(wait = True)
    print(f"Building slope Tick list : {len(data['slope_ticks'])}") 
    return()



def get_slope():
    global data
    data['slope_ticks'].popleft()
    data['slope_asks'].popleft()
    data['slope_bids'].popleft()
    
    data['slope_ticks'].append(data['price_tick'])
    data['slope_asks'].append(data['price_ask'])
    data['slope_bids'].append(data['price_bid'])
    num = len(data['slope_ticks'])
        
    l1 = []
    for i in range(num):
        l1.append(1 + ((i+1) * data['pip_size'] * 0.1))
    
    slope_y = list(data['slope_ticks'])[-num:]
    slope_x = l1
    data['slope_tick'], intercept, _, _, _ = linregress(slope_x, slope_y)
    data['slope_tick'] = math.degrees(math.atan(data['slope_tick']))
    
    if data['slope_tick'] > data['min_angle']:
        data['slope_type'] = 'long'
    elif data['slope_tick'] < -data['min_angle']:
        data['slope_type'] = 'short'
    
    return()

#--------------------------------------------------------------------------------------------------------------------------
def run_engine():  
    global data
    
    check_for_open_orders()

    for i, resp in enumerate(response_stream): 
        data['iter'] = i

        #Iteration Control ------------------------------------------
        if data['iter_limit_flag'] and i >= data['iter_max']:
            raise ValueError('Iter limit reached')

        if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
            continue

        # Date and Data ---------------------------------------------
        get_date_time(resp)
        if data['time_diff'] > data['max_time_diff']:
            raise ValueError('Time difference is above limit')

        get_prices(resp)

        # Build RSI ---------------------------------------------
        if len(data['list_tick']) < data['rsi_len']:
            before_rsi_len()          
            continue
        elif len(data['list_tick']) == data['rsi_len']:
            after_rsi_len()        

        # Find Slope ---------------------------------------------
        if len(data['slope_ticks']) < data['slope_len']:
            before_slope_len()
            continue
        elif len(data['list_tick']) == data['rsi_len']:
            get_slope()              
            
        check_for_open_orders()
        check_rsi_status()

        make_order()    

        take_profit()

        #make_hedge()

        # Reporting ---------------------------------------------
        data_rec = dict_to_tempdf()
        print(data_rec.head().transpose())    
        display.clear_output(wait = True)
    return()
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def reset_data():
    global data
    #Parameters ------------------------------------------
    data['order_num'] = 1
    data['leverage'] = 1
    
    data['order_num'] = data['order_num'] * data['leverage']

    data['val_profit_pip'] = 0.55
    data['val_stop_loss'] = 10    
    data['val_start_hedge'] = 5

    data['rsi_len'] = 14
    data['rsi_ob'] = 70
    data['rsi_os'] = 30
    
    data['slope_len'] = 100
    data['min_angle'] = 5
    
    data['iter_limit_flag'] = False
    data['iter_max'] = 600
    data['iter'] = 0
    data['max_time_diff'] = 15
        
        
    #Declaration ------------------------------------------

    data['ts_date_val'] = 0
    data['ts_time_val'] = 0
    
    data['price_ask'] = 0
    data['price_bid'] = 0
    data['price_stop'] = 0
    data['price_tick'] = 0

    data['slope_tick'] = None
    data['slope_type'] = None
    
    data['price_order_ask'] = 0
    data['price_order_bid'] = 0
    data['price_spread'] = 0
    
    data['rsi_val'] = 0
    data['rsi_status'] = None
    
    data['long_profit_val']      = 0
    data['long_max_profit']      = 0
    data['long_buffer_profit']   = 0
    
    data['short_profit_val']      = 0
    data['short_max_profit']      = 0
    data['short_buffer_profit']   = 0    

    data['profit_val_both'] = 0
    
    data['order_create'] = False
    data['order_current_open'] = False
    data['positions_info'] = None
    data['positions_long'] = 0
    data['positions_short'] = 0

    data['response_order'] = 0
    data['response_close'] = 0

    
    data['rsi_max'] = 0
    data['rsi_min'] = 100
    
    data['pip_take_profit'] = data['val_profit_pip'] * data['pip_size']    
    data['pip_stop_loss'] = data['val_stop_loss'] * data['pip_size']
    data['pip_start_hedge'] = data['val_start_hedge'] * data['pip_size']
    
    return()
#==========================================================================================================================

#### API Setup

In [8]:
data = {}
data_df = pd.DataFrame()

data['instrument'] = "EUR_USD"
data['pip_size'] = currs['currs'][data['instrument']]['pip_size']

data['icount'] = 0
data['start_ts'] = None

data['list_tick'] = collections.deque([])
data['list_up'] = collections.deque([])
data['list_down'] = collections.deque([])
data['list_AvgGain'] = collections.deque([])
data['list_Avgloss'] = collections.deque([])
data['list_RS'] = collections.deque([])
data['list_RSI'] = collections.deque([])
data['slope_ticks'] = collections.deque([])
data['slope_asks'] = collections.deque([])
data['slope_bids'] = collections.deque([])

access_token = config['oanda_demo_hedge']['token']
accountID = config['oanda_demo_hedge']['account_id']
params = {'instruments': data['instrument']}

api = API(access_token = access_token)
request_stream_data = pricing.PricingStream(accountID=accountID, params=params)
response_stream = api.request(request_stream_data)

## Core Engine

In [ ]:
%time check_for_open_orders()
%time check_for_open_orders()
%time check_for_open_orders()

run_flg = True 
reset_data()
data['start_ts'] = datetime.now().strftime("%Y-%b-%d, %I:%M:%S (%p)")

while run_flg ==  True:
    try:        
        run_engine()
    except:
        winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)
        data['icount'] = data['icount'] + 1

                                                                     0
iter                                                              1548
start_ts                                    2020-Nov-23, 01:22:44 (PM)
tot_ts                                      2020-Nov-23, 02:37:42 (PM)
time_diff                                                     0.190837
icount                                                              11
instrument                                                     EUR_USD
pip_size                                                        0.0001
order_num                                                            1
val_profit_pip                                                    0.55
val_stop_loss                                                       10
val_start_hedge                                                      5
rsi_len                                                             14
rsi_ob                                                              70
rsi_os

In [2]:
def get_weights(s):
    n = 1/s
    val = 0
    val_list = []
    for i in range(s):
        val += n
        val_list.append(val)
    val_list = np.array(val_list)
    tot = sum(val_list)
    val_list = val_list/tot
    return(val_list)

In [7]:
l = get_weights(20)
print(sum(l))
l

1.0


array([0.0047619 , 0.00952381, 0.01428571, 0.01904762, 0.02380952,
       0.02857143, 0.03333333, 0.03809524, 0.04285714, 0.04761905,
       0.05238095, 0.05714286, 0.06190476, 0.06666667, 0.07142857,
       0.07619048, 0.08095238, 0.08571429, 0.09047619, 0.0952381 ])

In [110]:
l = [1,2,3,4,5,6,7,8,9,10]

In [135]:
l = [22.2734,22.194,22.0847,22.1741,22.184,22.1344,22.2337,22.4323,22.2436,20,30,40,50,60,70,80,90,100,110,120,30,30,30,30,30,30,30,30,30,30]

In [136]:
ema_ready = False

In [137]:
def get_initial_ema(l, window_len):
    global ema_ready
    
    if not ema_ready:
        window_list = l[:window_len]
        sma = np.mean(window_list)
        smoothing_const = 2/(window_len + 1)
        ema = sma
        ema_ready = True
        print(f'sma : {sma}')
    if ema_ready:
        for i, val in enumerate(l):
            j = i + window_len
            new_list = l[(j-window_len+1):(j+1)]
            if len(new_list) == window_len:
                ema = (smoothing_const * (l[j]-ema)) + ema
                print(ema)
            
            

    
    return()

## Actions

 - Get the loop out and make sure we calculate ema only once in an iteration 
 - Global dict vars

In [138]:
get_initial_ema(l, 10)

sma : 21.995420000000003
23.450798181818183
26.45974396694215
30.73979051840721
36.059828605969535
42.23076885942962
49.09790179407878
56.534646922428095
64.43743839107754
72.72154050179071
81.31762404691968
71.98714694747973
64.35312022975614
58.10709836980048
52.99671684801858
48.815495602924294
45.394496402392605
42.59549705650304
40.3054066825934
38.431696376667325
36.89866067181872


()

In [114]:
l[-5:]

[6, 7, 8, 9, 10]

In [9]:
for i, resp in enumerate(response_stream): 


    if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        continue
    else:
        print(json.dumps(resp, indent=4, sort_keys=True))


{
    "asks": [
        {
            "liquidity": 1000000,
            "price": "1.21929"
        },
        {
            "liquidity": 2000000,
            "price": "1.21931"
        },
        {
            "liquidity": 2000000,
            "price": "1.21932"
        },
        {
            "liquidity": 5000000,
            "price": "1.21933"
        }
    ],
    "bids": [
        {
            "liquidity": 1000000,
            "price": "1.21922"
        },
        {
            "liquidity": 2000000,
            "price": "1.21921"
        },
        {
            "liquidity": 2000000,
            "price": "1.21920"
        },
        {
            "liquidity": 5000000,
            "price": "1.21918"
        }
    ],
    "closeoutAsk": "1.21933",
    "closeoutBid": "1.21918",
    "instrument": "EUR_USD",
    "status": "tradeable",
    "time": "2020-12-23T07:34:38.773745668Z",
    "tradeable": true,
    "type": "PRICE"
}
{
    "asks": [
        {
            "liquidity": 1000000,
   

{
    "asks": [
        {
            "liquidity": 1000000,
            "price": "1.21925"
        },
        {
            "liquidity": 2000000,
            "price": "1.21927"
        },
        {
            "liquidity": 2000000,
            "price": "1.21928"
        },
        {
            "liquidity": 5000000,
            "price": "1.21929"
        }
    ],
    "bids": [
        {
            "liquidity": 1000000,
            "price": "1.21917"
        },
        {
            "liquidity": 2000000,
            "price": "1.21916"
        },
        {
            "liquidity": 2000000,
            "price": "1.21915"
        },
        {
            "liquidity": 5000000,
            "price": "1.21913"
        }
    ],
    "closeoutAsk": "1.21929",
    "closeoutBid": "1.21913",
    "instrument": "EUR_USD",
    "status": "tradeable",
    "time": "2020-12-23T07:34:58.577019322Z",
    "tradeable": true,
    "type": "PRICE"
}
{
    "asks": [
        {
            "liquidity": 1000000,
   

{
    "asks": [
        {
            "liquidity": 1000000,
            "price": "1.21919"
        },
        {
            "liquidity": 2000000,
            "price": "1.21921"
        },
        {
            "liquidity": 2000000,
            "price": "1.21922"
        },
        {
            "liquidity": 5000000,
            "price": "1.21923"
        }
    ],
    "bids": [
        {
            "liquidity": 1000000,
            "price": "1.21911"
        },
        {
            "liquidity": 2000000,
            "price": "1.21910"
        },
        {
            "liquidity": 2000000,
            "price": "1.21909"
        },
        {
            "liquidity": 5000000,
            "price": "1.21907"
        }
    ],
    "closeoutAsk": "1.21923",
    "closeoutBid": "1.21907",
    "instrument": "EUR_USD",
    "status": "tradeable",
    "time": "2020-12-23T07:35:10.318773886Z",
    "tradeable": true,
    "type": "PRICE"
}
{
    "asks": [
        {
            "liquidity": 1000000,
   

KeyboardInterrupt: 